In [40]:
#Import packages
import pandas as pd
import numpy as np
import sqlite3
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [41]:
#Importing the Database and determining what tables are in the database
con = sqlite3.connect('acc1819.db')
cursor = con.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cursor.fetchall())

[('games',), ('box_scores',)]


In [42]:
#Creating dataframes from the 2 tables
games = pd.read_sql_query("SELECT * FROM games", con)
boxscores = pd.read_sql_query("SELECT * FROM box_scores", con)

In [43]:
games.head()

,GameId,GameDate,NeutralSite,AwayTeam,HomeTeam
0,1,1/1/2019 13:00,0,Notre Dame Fighting Irish,Virginia Tech Hokies
1,2,1/3/2019 19:00,0,North Carolina State Wolfpack,Miami (FL) Hurricanes
2,3,1/5/2019 3:27,0,Clemson Tigers,Duke Blue Devils
3,4,1/5/2019 12:00,0,Boston College Eagles,Virginia Tech Hokies
4,5,1/5/2019 12:00,0,Syracuse Orange,Notre Dame Fighting Irish


In [44]:
boxscores.head()

,GameId,Team,Home,Score,AST,TOV,STL,BLK,Rebounds,ORB,DRB,FGA,FGM,3FGM,3FGA,FTA,FTM,Fouls
0,1,Virginia Tech Hokies,1,81,19,7,5,1,24,2,22,55,33,11,18,5,4,13
1,1,Notre Dame Fighting Irish,0,66,13,11,2,5,30,13,17,56,23,13,34,13,7,10
2,2,North Carolina State Wolfpack,0,87,17,16,4,3,50,17,33,68,31,11,30,18,14,23
3,2,Miami (FL) Hurricanes,1,82,12,7,7,1,27,9,18,61,28,10,25,29,16,14
4,3,Duke Blue Devils,1,87,16,12,13,6,39,12,27,67,32,7,23,21,16,15


In [45]:
#Converting the dataframes to csv's to view the entire dataset and vizualize in Tableau
games.to_csv("Games.csv")
boxscores.to_csv("BoxScores.csv")

In [46]:
#Added Win Column, Neutral Site Column and Margin Columns in Excel,then read file back in
boxscores = pd.read_csv("BoxScores_WithMargins.csv")
boxscores.head()

,Unnamed: 0,GameId,Team,Home,Score,AST,TOV,STL,BLK,Rebounds,...,STL_Margin,BLK_Margin,Rebounds_Margin,ORB_Margin,DRB_Margin,Fouls_Margin,FGP_Margin,3FGP_Margin,FTP_Margin,Neutral_Indicator
0,0,1,Virginia Tech Hokies,1,81,19,7,5,1,24,...,3,-4,-6,-11,5,3,0.189286,0.228758,0.261538,0
1,1,1,Notre Dame Fighting Irish,0,66,13,11,2,5,30,...,-3,4,6,11,-5,-3,-0.189286,-0.228758,-0.261538,0
2,2,2,North Carolina State Wolfpack,0,87,17,16,4,3,50,...,-3,2,23,8,15,9,-0.003134,-0.033333,0.226054,0
3,3,2,Miami (FL) Hurricanes,1,82,12,7,7,1,27,...,3,-2,-23,-8,-15,-9,0.003134,0.033333,-0.226054,0
4,4,3,Duke Blue Devils,1,87,16,12,13,6,39,...,7,4,4,3,1,-1,0.049041,-0.095652,0.095238,0


In [47]:
#Drop Unnamed Column
boxscores = boxscores.drop(["Unnamed: 0"], axis=1)
boxscores.head()

,GameId,Team,Home,Score,AST,TOV,STL,BLK,Rebounds,ORB,...,STL_Margin,BLK_Margin,Rebounds_Margin,ORB_Margin,DRB_Margin,Fouls_Margin,FGP_Margin,3FGP_Margin,FTP_Margin,Neutral_Indicator
0,1,Virginia Tech Hokies,1,81,19,7,5,1,24,2,...,3,-4,-6,-11,5,3,0.189286,0.228758,0.261538,0
1,1,Notre Dame Fighting Irish,0,66,13,11,2,5,30,13,...,-3,4,6,11,-5,-3,-0.189286,-0.228758,-0.261538,0
2,2,North Carolina State Wolfpack,0,87,17,16,4,3,50,17,...,-3,2,23,8,15,9,-0.003134,-0.033333,0.226054,0
3,2,Miami (FL) Hurricanes,1,82,12,7,7,1,27,9,...,3,-2,-23,-8,-15,-9,0.003134,0.033333,-0.226054,0
4,3,Duke Blue Devils,1,87,16,12,13,6,39,12,...,7,4,4,3,1,-1,0.049041,-0.095652,0.095238,0


In [48]:
#Creating a function to summarize the dataframe
def summarize_dataframe(df):
    """Summarize a dataframe, and report missing values."""
    missing_values = pd.DataFrame({'Variable Name': df.columns,
                                   'Data Type': df.dtypes,
                                   'Missing Values': df.isnull().sum(),
                                   'Unique Values': [df[name].nunique() for name in df.columns]}
                                 ).set_index('Variable Name')
    with pd.option_context("display.max_rows", 1000):
        display(pd.concat([missing_values, df.describe(include='all').transpose()], axis=1).fillna(""))

In [49]:
summarize_dataframe(boxscores)

,Data Type,Missing Values,Unique Values,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Variable Name,,,,,,,,,,,,,,
GameId,int64,0,149,298.0,,,,75,43.084,1,38,75,112,149
Team,object,0,15,298.0,15,Duke Blue Devils,21,,,,,,,
Home,int64,0,2,298.0,,,,0.5,0.500841,0,0,0.5,1,1
Score,int64,0,57,298.0,,,,69.4161,12.0678,24,61.25,69,78,113
AST,int64,0,25,298.0,,,,12.9799,4.37389,2,10,13,16,26
TOV,int64,0,22,298.0,,,,11.6611,3.63335,2,9,11.5,14,23
STL,int64,0,16,298.0,,,,6.03356,2.74608,0,4,6,8,15
BLK,int64,0,14,298.0,,,,3.72483,2.46008,0,2,3,5,13
Rebounds,int64,0,34,298.0,,,,32.9295,6.5236,18,28,32,37,53


In [50]:
#Aggregating data by team and getting the average for all stats
team_agg = boxscores.groupby(['Team'], as_index=False).agg('mean')
team_agg.head()

,Team,GameId,Home,Score,AST,TOV,STL,BLK,Rebounds,ORB,...,STL_Margin,BLK_Margin,Rebounds_Margin,ORB_Margin,DRB_Margin,Fouls_Margin,FGP_Margin,3FGP_Margin,FTP_Margin,Neutral_Indicator
0,Boston College Eagles,74.789474,0.526316,66.736842,12.210526,10.947368,4.578947,3.000000,32.315789,9.473684,...,-1.631579,-0.473684,-4.315789,-1.631579,-2.684211,0.315789,-0.035284,-0.007591,-0.064392,0.052632
1,Clemson Tigers,74.315789,0.473684,64.736842,12.315789,12.157895,6.105263,4.368421,32.789474,7.473684,...,0.842105,1.000000,2.368421,0.105263,2.263158,1.052632,0.042511,0.004203,-0.014928,0.052632
2,Duke Blue Devils,79.571429,0.523810,78.523810,14.619048,12.809524,8.190476,6.190476,36.619048,10.761905,...,1.857143,3.047619,3.619048,0.428571,3.190476,-0.952381,0.054091,-0.009489,0.012010,0.142857
3,Florida State Seminoles,80.904762,0.476190,70.952381,13.000000,12.000000,5.904762,4.666667,35.142857,9.619048,...,0.666667,1.904762,4.809524,2.333333,2.476190,0.952381,0.026399,0.011274,-0.000204,0.142857
4,Georgia Tech Yellow Jackets,70.421053,0.526316,61.263158,12.631579,13.368421,6.368421,5.157895,28.842105,6.315789,...,-0.421053,1.263158,-5.052632,-4.000000,-1.052632,0.210526,0.022629,-0.020608,-0.089359,0.052632


In [51]:
#Setting Team as the index and identifying the columns I don't want changed in the boxscores df for the model
boxscores.drop(boxscores.columns.difference(["Team","GameId","Win","Home","Neutral_Indicator"]), 1, inplace=True)
team_agg.set_index('Team', inplace = True)
team_agg.head()

,GameId,Home,Score,AST,TOV,STL,BLK,Rebounds,ORB,DRB,...,STL_Margin,BLK_Margin,Rebounds_Margin,ORB_Margin,DRB_Margin,Fouls_Margin,FGP_Margin,3FGP_Margin,FTP_Margin,Neutral_Indicator
Team,,,,,,,,,,,,,,,,,,,,,
Boston College Eagles,74.789474,0.526316,66.736842,12.210526,10.947368,4.578947,3.000000,32.315789,9.473684,22.842105,...,-1.631579,-0.473684,-4.315789,-1.631579,-2.684211,0.315789,-0.035284,-0.007591,-0.064392,0.052632
Clemson Tigers,74.315789,0.473684,64.736842,12.315789,12.157895,6.105263,4.368421,32.789474,7.473684,25.315789,...,0.842105,1.000000,2.368421,0.105263,2.263158,1.052632,0.042511,0.004203,-0.014928,0.052632
Duke Blue Devils,79.571429,0.523810,78.523810,14.619048,12.809524,8.190476,6.190476,36.619048,10.761905,25.857143,...,1.857143,3.047619,3.619048,0.428571,3.190476,-0.952381,0.054091,-0.009489,0.012010,0.142857
Florida State Seminoles,80.904762,0.476190,70.952381,13.000000,12.000000,5.904762,4.666667,35.142857,9.619048,25.523810,...,0.666667,1.904762,4.809524,2.333333,2.476190,0.952381,0.026399,0.011274,-0.000204,0.142857
Georgia Tech Yellow Jackets,70.421053,0.526316,61.263158,12.631579,13.368421,6.368421,5.157895,28.842105,6.315789,22.526316,...,-0.421053,1.263158,-5.052632,-4.000000,-1.052632,0.210526,0.022629,-0.020608,-0.089359,0.052632


In [52]:
#Dropping the columns I don't want updated in the df for the model
team_agg.drop(columns = ["GameId","Win","Home","Neutral_Indicator"], inplace = True)
team_agg.head()

,Score,AST,TOV,STL,BLK,Rebounds,ORB,DRB,FGA,FGM,...,TOV_Margin,STL_Margin,BLK_Margin,Rebounds_Margin,ORB_Margin,DRB_Margin,Fouls_Margin,FGP_Margin,3FGP_Margin,FTP_Margin
Team,,,,,,,,,,,,,,,,,,,,,
Boston College Eagles,66.736842,12.210526,10.947368,4.578947,3.000000,32.315789,9.473684,22.842105,58.894737,24.000000,...,0.315789,-1.631579,-0.473684,-4.315789,-1.631579,-2.684211,0.315789,-0.035284,-0.007591,-0.064392
Clemson Tigers,64.736842,12.315789,12.157895,6.105263,4.368421,32.789474,7.473684,25.315789,54.631579,23.368421,...,0.210526,0.842105,1.000000,2.368421,0.105263,2.263158,1.052632,0.042511,0.004203,-0.014928
Duke Blue Devils,78.523810,14.619048,12.809524,8.190476,6.190476,36.619048,10.761905,25.857143,61.666667,28.571429,...,0.285714,1.857143,3.047619,3.619048,0.428571,3.190476,-0.952381,0.054091,-0.009489,0.012010
Florida State Seminoles,70.952381,13.000000,12.000000,5.904762,4.666667,35.142857,9.619048,25.523810,57.000000,24.238095,...,0.047619,0.666667,1.904762,4.809524,2.333333,2.476190,0.952381,0.026399,0.011274,-0.000204
Georgia Tech Yellow Jackets,61.263158,12.631579,13.368421,6.368421,5.157895,28.842105,6.315789,22.526316,52.157895,22.526316,...,2.894737,-0.421053,1.263158,-5.052632,-4.000000,-1.052632,0.210526,0.022629,-0.020608,-0.089359


In [53]:
#Normalizing the aggregated average stats
teamagg_norm = (team_agg-team_agg.mean())/team_agg.std()
teamagg_norm.head()

,Score,AST,TOV,STL,BLK,Rebounds,ORB,DRB,FGA,FGM,...,TOV_Margin,STL_Margin,BLK_Margin,Rebounds_Margin,ORB_Margin,DRB_Margin,Fouls_Margin,FGP_Margin,3FGP_Margin,FTP_Margin
Team,,,,,,,,,,,,,,,,,,,,,
Boston College Eagles,-0.439083,-0.361431,-0.510828,-1.189060,-0.568715,-0.188703,0.178004,-0.386010,0.248952,-0.173362,...,0.213712,-1.245546,-0.256796,-0.999331,-0.881783,-0.808245,0.316352,-0.622449,-0.146260,-1.251287
Clemson Tigers,-0.780014,-0.310563,0.349148,0.071245,0.523698,-0.034661,-1.012700,0.699984,-0.761551,-0.431511,...,0.138165,0.668789,0.582241,0.567466,0.069002,0.699262,1.065326,0.771263,0.090829,-0.284750
Duke Blue Devils,1.570192,0.802456,0.812075,1.793041,1.978248,1.210715,0.944948,0.937648,0.905987,1.695139,...,0.192127,1.454306,1.748045,0.860616,0.245987,0.981824,-0.972698,0.978718,-0.184413,0.241640
Florida State Seminoles,0.279523,0.020072,0.236977,-0.094312,0.761788,0.730659,0.264546,0.791308,-0.200160,-0.076044,...,0.021249,0.533020,1.097363,1.139667,1.288694,0.764175,0.963425,0.482614,0.232979,0.002963
Georgia Tech Yellow Jackets,-1.372159,-0.157962,1.209124,0.288539,1.153936,-1.318343,-1.702054,-0.524648,-1.347892,-0.775708,...,2.064591,-0.308744,0.732069,-1.172049,-2.178306,-0.311088,0.209356,0.415066,-0.407947,-1.739144


In [54]:
#Putting the normalized averages for all the teams and merging with the boxscores df
boxscores_norm = pd.merge(boxscores, teamagg_norm, on="Team", how = "left")
boxscores_norm.head()

,GameId,Team,Home,Win,Neutral_Indicator,Score,AST,TOV,STL,BLK,...,TOV_Margin,STL_Margin,BLK_Margin,Rebounds_Margin,ORB_Margin,DRB_Margin,Fouls_Margin,FGP_Margin,3FGP_Margin,FTP_Margin
0,1,Virginia Tech Hokies,1,1,0,0.031941,0.382500,-0.757604,-0.098244,-1.406932,...,-0.981805,0.597510,-0.499517,0.117783,-0.371817,0.360073,-0.766985,0.620415,0.394431,0.363175
1,1,Notre Dame Fighting Irish,0,0,0,-1.076086,-0.583974,-1.858747,-0.841390,-0.129649,...,-0.300002,0.055794,0.098297,-1.194872,-0.700269,-1.163471,-0.563692,-1.324409,-0.739654,0.965842
2,2,North Carolina State Wolfpack,0,1,0,0.671188,0.140882,0.130415,0.727473,-0.009904,...,-0.874152,0.790980,-0.442582,-0.198661,1.024108,-0.828291,2.536524,-0.665758,0.347705,0.423812
3,2,Miami (FL) Hurricanes,1,0,0,-0.172618,-0.439003,-0.757604,0.397186,-1.207356,...,-1.232996,0.674898,-0.300246,-1.171432,-0.289703,-1.361532,-1.630979,-0.836985,-1.196096,-0.271763
4,3,Duke Blue Devils,1,1,0,1.570192,0.802456,0.812075,1.793041,1.978248,...,0.192127,1.454306,1.748045,0.860616,0.245987,0.981824,-0.972698,0.978718,-0.184413,0.241640


In [55]:
#Splitting the dataframe for home team
boxscores_home = boxscores_norm[boxscores_norm["Home"] > 0]
boxscores_home.head()

,GameId,Team,Home,Win,Neutral_Indicator,Score,AST,TOV,STL,BLK,...,TOV_Margin,STL_Margin,BLK_Margin,Rebounds_Margin,ORB_Margin,DRB_Margin,Fouls_Margin,FGP_Margin,3FGP_Margin,FTP_Margin
0,1,Virginia Tech Hokies,1,1,0,0.031941,0.382500,-0.757604,-0.098244,-1.406932,...,-0.981805,0.597510,-0.499517,0.117783,-0.371817,0.360073,-0.766985,0.620415,0.394431,0.363175
3,2,Miami (FL) Hurricanes,1,0,0,-0.172618,-0.439003,-0.757604,0.397186,-1.207356,...,-1.232996,0.674898,-0.300246,-1.171432,-0.289703,-1.361532,-1.630979,-0.836985,-1.196096,-0.271763
4,3,Duke Blue Devils,1,1,0,1.570192,0.802456,0.812075,1.793041,1.978248,...,0.192127,1.454306,1.748045,0.860616,0.245987,0.981824,-0.972698,0.978718,-0.184413,0.241640
6,4,Virginia Tech Hokies,1,1,0,0.031941,0.382500,-0.757604,-0.098244,-1.406932,...,-0.981805,0.597510,-0.499517,0.117783,-0.371817,0.360073,-0.766985,0.620415,0.394431,0.363175
8,5,Notre Dame Fighting Irish,1,0,0,-1.076086,-0.583974,-1.858747,-0.841390,-0.129649,...,-0.300002,0.055794,0.098297,-1.194872,-0.700269,-1.163471,-0.563692,-1.324409,-0.739654,0.965842


In [56]:
#Splitting the data frame for away teams
boxscores_away = boxscores_norm[boxscores_norm["Home"] < 1]
boxscores_away.head()

,GameId,Team,Home,Win,Neutral_Indicator,Score,AST,TOV,STL,BLK,...,TOV_Margin,STL_Margin,BLK_Margin,Rebounds_Margin,ORB_Margin,DRB_Margin,Fouls_Margin,FGP_Margin,3FGP_Margin,FTP_Margin
1,1,Notre Dame Fighting Irish,0,0,0,-1.076086,-0.583974,-1.858747,-0.841390,-0.129649,...,-0.300002,0.055794,0.098297,-1.194872,-0.700269,-1.163471,-0.563692,-1.324409,-0.739654,0.965842
2,2,North Carolina State Wolfpack,0,1,0,0.671188,0.140882,0.130415,0.727473,-0.009904,...,-0.874152,0.790980,-0.442582,-0.198661,1.024108,-0.828291,2.536524,-0.665758,0.347705,0.423812
5,3,Clemson Tigers,0,0,0,-0.780014,-0.310563,0.349148,0.071245,0.523698,...,0.138165,0.668789,0.582241,0.567466,0.069002,0.699262,1.065326,0.771263,0.090829,-0.284750
7,4,Boston College Eagles,0,0,0,-0.439083,-0.361431,-0.510828,-1.189060,-0.568715,...,0.213712,-1.245546,-0.256796,-0.999331,-0.881783,-0.808245,0.316352,-0.622449,-0.146260,-1.251287
9,5,Syracuse Orange,0,1,0,-0.138524,-0.632297,0.521143,1.594475,1.027888,...,-1.591840,0.558816,0.895381,-0.819828,-0.618156,-0.721643,-0.563692,0.197364,-0.051161,-0.991965


In [57]:
#Creating combined rows for each matchup based on GameId
boxscores_comb = pd.merge(boxscores_home, boxscores_away, on="GameId", how = "left")
boxscores_comb.head()

,GameId,Team_x,Home_x,Win_x,Neutral_Indicator_x,Score_x,AST_x,TOV_x,STL_x,BLK_x,...,TOV_Margin_y,STL_Margin_y,BLK_Margin_y,Rebounds_Margin_y,ORB_Margin_y,DRB_Margin_y,Fouls_Margin_y,FGP_Margin_y,3FGP_Margin_y,FTP_Margin_y
0,1,Virginia Tech Hokies,1,1,0,0.031941,0.382500,-0.757604,-0.098244,-1.406932,...,-0.300002,0.055794,0.098297,-1.194872,-0.700269,-1.163471,-0.563692,-1.324409,-0.739654,0.965842
1,2,Miami (FL) Hurricanes,1,0,0,-0.172618,-0.439003,-0.757604,0.397186,-1.207356,...,-0.874152,0.790980,-0.442582,-0.198661,1.024108,-0.828291,2.536524,-0.665758,0.347705,0.423812
2,3,Duke Blue Devils,1,1,0,1.570192,0.802456,0.812075,1.793041,1.978248,...,0.138165,0.668789,0.582241,0.567466,0.069002,0.699262,1.065326,0.771263,0.090829,-0.284750
3,4,Virginia Tech Hokies,1,1,0,0.031941,0.382500,-0.757604,-0.098244,-1.406932,...,0.213712,-1.245546,-0.256796,-0.999331,-0.881783,-0.808245,0.316352,-0.622449,-0.146260,-1.251287
4,5,Notre Dame Fighting Irish,1,0,0,-1.076086,-0.583974,-1.858747,-0.841390,-0.129649,...,-1.591840,0.558816,0.895381,-0.819828,-0.618156,-0.721643,-0.563692,0.197364,-0.051161,-0.991965


In [58]:
#Get a list of columns in new dataframe
boxscores_comb.columns

Index(['GameId', 'Team_x', 'Home_x', 'Win_x', 'Neutral_Indicator_x', 'Score_x',
       'AST_x', 'TOV_x', 'STL_x', 'BLK_x', 'Rebounds_x', 'ORB_x', 'DRB_x',
       'FGA_x', 'FGM_x', '3FGM_x', '3FGA_x', 'FTA_x', 'FTM_x', 'Fouls_x',
       'FGP_x', '3FGP_x', 'FTP_x', 'Score_Margin_x', 'AST_Margin_x',
       'TOV_Margin_x', 'STL_Margin_x', 'BLK_Margin_x', 'Rebounds_Margin_x',
       'ORB_Margin_x', 'DRB_Margin_x', 'Fouls_Margin_x', 'FGP_Margin_x',
       '3FGP_Margin_x', 'FTP_Margin_x', 'Team_y', 'Home_y', 'Win_y',
       'Neutral_Indicator_y', 'Score_y', 'AST_y', 'TOV_y', 'STL_y', 'BLK_y',
       'Rebounds_y', 'ORB_y', 'DRB_y', 'FGA_y', 'FGM_y', '3FGM_y', '3FGA_y',
       'FTA_y', 'FTM_y', 'Fouls_y', 'FGP_y', '3FGP_y', 'FTP_y',
       'Score_Margin_y', 'AST_Margin_y', 'TOV_Margin_y', 'STL_Margin_y',
       'BLK_Margin_y', 'Rebounds_Margin_y', 'ORB_Margin_y', 'DRB_Margin_y',
       'Fouls_Margin_y', 'FGP_Margin_y', '3FGP_Margin_y', 'FTP_Margin_y'],
      dtype='object')

In [59]:
#Rename Columns
boxscores_comb = boxscores_comb.rename(columns = {'Team_x': 'Team1', 'Home_x': 'Home_Team1', 'Win_x': 'Win_Team1', 
                                          'Neutral_Indicator_x': 'Neutral_Indicator_Team1', 'Score_x': 'Score_Team1', 
                                          'AST_x': 'AST_Team1', 
                                          'TOV_x': 'TOV_Team1', 'STL_x': 'STL_Team1', 'BLK_x': 'BLK_Team1', 
                                          'Rebounds_x': 'Rebounds_Team1', 'ORB_x': 'ORB_Team1', 'DRB_x': 'DRB_Team1', 
                                          'FGA_x': 'FGA_Team1', 'FGM_x': 'FGM_Team1', '3FGM_x': '3FGM_Team1', 
                                          '3FGA_x': '3FGA_Team1', 'FTA_x': 'FTA_Team1', 'FTM_x': 'FTM_Team1', 
                                          'Fouls_x': 'Fouls_Team1', 'FGP_x': 'FGP_Team1', '3FGP_x': '3FGP__Team1', 
                                          'FTP_x': 'FTP_Team1', 'Score_Margin_x': 'Score_Margin_Team1', 
                                          'AST_Margin_x': 'AST_Margin_Team1', 
                                          'TOV_Margin_x': 'TOV_Margin_Team1', 'STL_Margin_x': 'STL_Margin_Team1', 
                                          'BLK_Margin_x': 'BLK_Margin_Team1', 
                                          'Rebounds_Margin_x': 'Rebounds_Margin_Team1', 'ORB_Margin_x': 'ORB_Margin_Team1',
                                          'DRB_Margin_x': 'DRB_Margin_Team1', 
                                          'Fouls_Margin_x': 'Fouls_Margin_Team1', 'FGP_Margin_x': 'FGP_Margin_Team1', 
                                          '3FGP_Margin_x': '3FGP_Margin_Team1', 
                                          'FTP_Margin_x': 'FTP_Margin_Team1', 
                                         'Team_y': 'Team2', 'Home_y': 'Home_Team2', 'Win_y': 'Win_Team2', 
                                          'Neutral_Indicator_y': 'Neutral_Indicator_Team2', 'Score_y': 'Score_Team2', 
                                          'AST_y': 'AST_Team2', 
                                          'TOV_y': 'TOV_Team2', 'STL_y': 'STL_Team2', 'BLK_y': 'BLK_Team2', 
                                          'Rebounds_y': 'Rebounds_Team2', 'ORB_y': 'ORB_Team2', 'DRB_y': 'DRB_Team2', 
                                          'FGA_y': 'FGA_Team2', 'FGM_y': 'FGM_Team2', '3FGM_y': '3FGM_Team2', 
                                          '3FGA_y': '3FGA_Team2', 'FTA_y': 'FTA_Team2', 'FTM_y': 'FTM_Team2', 
                                          'Fouls_y': 'Fouls_Team2', 'FGP_y': 'FGP_Team2', '3FGP_y': '3FGP__Team2', 
                                          'FTP_y': 'FTP_Team2', 'Score_Margin_y': 'Score_Margin_Team2', 
                                          'AST_Margin_y': 'AST_Margin_Team2', 
                                          'TOV_Margin_y': 'TOV_Margin_Team2', 'STL_Margin_y': 'STL_Margin_Team2', 
                                          'BLK_Margin_y': 'BLK_Margin_Team2', 
                                          'Rebounds_Margin_y': 'Rebounds_Margin_Team2', 'ORB_Margin_y': 'ORB_Margin_Team2',
                                          'DRB_Margin_y': 'DRB_Margin_Team2', 
                                          'Fouls_Margin_y': 'Fouls_Margin_Team2', 'FGP_Margin_y': 'FGP_Margin_Team2', 
                                          '3FGP_Margin_y': '3FGP_Margin_Team2', 
                                          'FTP_Margin_y': 'FTP_Margin_Team2'}, inplace = False)

boxscores_comb.head()

,GameId,Team1,Home_Team1,Win_Team1,Neutral_Indicator_Team1,Score_Team1,AST_Team1,TOV_Team1,STL_Team1,BLK_Team1,...,TOV_Margin_Team2,STL_Margin_Team2,BLK_Margin_Team2,Rebounds_Margin_Team2,ORB_Margin_Team2,DRB_Margin_Team2,Fouls_Margin_Team2,FGP_Margin_Team2,3FGP_Margin_Team2,FTP_Margin_Team2
0,1,Virginia Tech Hokies,1,1,0,0.031941,0.382500,-0.757604,-0.098244,-1.406932,...,-0.300002,0.055794,0.098297,-1.194872,-0.700269,-1.163471,-0.563692,-1.324409,-0.739654,0.965842
1,2,Miami (FL) Hurricanes,1,0,0,-0.172618,-0.439003,-0.757604,0.397186,-1.207356,...,-0.874152,0.790980,-0.442582,-0.198661,1.024108,-0.828291,2.536524,-0.665758,0.347705,0.423812
2,3,Duke Blue Devils,1,1,0,1.570192,0.802456,0.812075,1.793041,1.978248,...,0.138165,0.668789,0.582241,0.567466,0.069002,0.699262,1.065326,0.771263,0.090829,-0.284750
3,4,Virginia Tech Hokies,1,1,0,0.031941,0.382500,-0.757604,-0.098244,-1.406932,...,0.213712,-1.245546,-0.256796,-0.999331,-0.881783,-0.808245,0.316352,-0.622449,-0.146260,-1.251287
4,5,Notre Dame Fighting Irish,1,0,0,-1.076086,-0.583974,-1.858747,-0.841390,-0.129649,...,-1.591840,0.558816,0.895381,-0.819828,-0.618156,-0.721643,-0.563692,0.197364,-0.051161,-0.991965


In [60]:
#Dropping the Team2 Win Column since we will be predicting on Team1 Win
boxscores_comb = boxscores_comb.drop("Win_Team2", axis=1)
boxscores_comb.head()

,GameId,Team1,Home_Team1,Win_Team1,Neutral_Indicator_Team1,Score_Team1,AST_Team1,TOV_Team1,STL_Team1,BLK_Team1,...,TOV_Margin_Team2,STL_Margin_Team2,BLK_Margin_Team2,Rebounds_Margin_Team2,ORB_Margin_Team2,DRB_Margin_Team2,Fouls_Margin_Team2,FGP_Margin_Team2,3FGP_Margin_Team2,FTP_Margin_Team2
0,1,Virginia Tech Hokies,1,1,0,0.031941,0.382500,-0.757604,-0.098244,-1.406932,...,-0.300002,0.055794,0.098297,-1.194872,-0.700269,-1.163471,-0.563692,-1.324409,-0.739654,0.965842
1,2,Miami (FL) Hurricanes,1,0,0,-0.172618,-0.439003,-0.757604,0.397186,-1.207356,...,-0.874152,0.790980,-0.442582,-0.198661,1.024108,-0.828291,2.536524,-0.665758,0.347705,0.423812
2,3,Duke Blue Devils,1,1,0,1.570192,0.802456,0.812075,1.793041,1.978248,...,0.138165,0.668789,0.582241,0.567466,0.069002,0.699262,1.065326,0.771263,0.090829,-0.284750
3,4,Virginia Tech Hokies,1,1,0,0.031941,0.382500,-0.757604,-0.098244,-1.406932,...,0.213712,-1.245546,-0.256796,-0.999331,-0.881783,-0.808245,0.316352,-0.622449,-0.146260,-1.251287
4,5,Notre Dame Fighting Irish,1,0,0,-1.076086,-0.583974,-1.858747,-0.841390,-0.129649,...,-1.591840,0.558816,0.895381,-0.819828,-0.618156,-0.721643,-0.563692,0.197364,-0.051161,-0.991965


In [61]:
#Selecting 5 independent variables, duplicated for team1 and team2, for logistic regression model
ind_var_selected = ['DRB_Margin_Team1', 'FGP_Margin_Team1', '3FGP_Margin_Team1', 'AST_Margin_Team1', 'Fouls_Margin_Team1', 
                   'DRB_Margin_Team2', 'FGP_Margin_Team2', '3FGP_Margin_Team2', 'AST_Margin_Team2', 'Fouls_Margin_Team2']

In [62]:
#Creating our X and y variables
X = boxscores_comb[ind_var_selected]
y = boxscores_comb['Win_Team1']

In [63]:
#Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=5)
print(X_train.shape)
print(X_test.shape)

(104, 10)
(45, 10)


In [64]:
#Logistic Regression Model
lr = LogisticRegression(C = 1e9, random_state=23)
lr.fit(X, y)
lr_pred = lr.predict_proba(X_test)[:,1] 

In [65]:
#Scoring metric for logistic regression
log_loss(y_test, lr_pred)

0.40317964558791525

In [66]:
#Determine the unique teams to create new dataframe
boxscores_comb.Team1.unique()

array(['Virginia Tech Hokies', 'Miami (FL) Hurricanes',
       'Duke Blue Devils', 'Notre Dame Fighting Irish',
       'Pittsburgh Panthers', 'Georgia Tech Yellow Jackets',
       'Virginia Cavaliers', 'Louisville Cardinals',
       'Wake Forest Demon Deacons', 'North Carolina State Wolfpack',
       'Syracuse Orange', 'Florida State Seminoles',
       'Boston College Eagles', 'Clemson Tigers',
       'North Carolina Tar Heels'], dtype=object)

In [67]:
#Creating Test Dataframe with 2 team columns that matches up every team against each other
testdf = pd.DataFrame({'Team' : ['Virginia Tech Hokies']*14 + ['Miami (FL) Hurricanes']*14 + 
                       ['Duke Blue Devils']*14 + ['Notre Dame Fighting Irish']*14 + 
                       ['Pittsburgh Panthers']*14 + ['Georgia Tech Yellow Jackets']*14 + 
                       ['Virginia Cavaliers']*14 + ['Louisville Cardinals']*14 + ['Wake Forest Demon Deacons']*14 + 
                       ['North Carolina State Wolfpack']*14 + ['Syracuse Orange']*14 + ['Florida State Seminoles']*14 +
                       ['Boston College Eagles']*14 + ['Clemson Tigers']*14 + ['North Carolina Tar Heels']*14, 
                      'Team2' : ['Miami (FL) Hurricanes'] + 
                       ['Duke Blue Devils'] + ['Notre Dame Fighting Irish'] + 
                       ['Pittsburgh Panthers'] + ['Georgia Tech Yellow Jackets'] + 
                       ['Virginia Cavaliers'] + ['Louisville Cardinals'] + ['Wake Forest Demon Deacons'] + 
                       ['North Carolina State Wolfpack'] + ['Syracuse Orange'] + ['Florida State Seminoles'] +
                       ['Boston College Eagles'] + ['Clemson Tigers'] + ['North Carolina Tar Heels'] + 
                      ['Virginia Tech Hokies'] + 
                       ['Duke Blue Devils'] + ['Notre Dame Fighting Irish'] + 
                       ['Pittsburgh Panthers'] + ['Georgia Tech Yellow Jackets'] + 
                       ['Virginia Cavaliers'] + ['Louisville Cardinals'] + ['Wake Forest Demon Deacons'] + 
                       ['North Carolina State Wolfpack'] + ['Syracuse Orange'] + ['Florida State Seminoles'] +
                       ['Boston College Eagles'] + ['Clemson Tigers'] + ['North Carolina Tar Heels'] + 
                      ['Miami (FL) Hurricanes'] + 
                       ['Virginia Tech Hokies'] + ['Notre Dame Fighting Irish'] + 
                       ['Pittsburgh Panthers'] + ['Georgia Tech Yellow Jackets'] + 
                       ['Virginia Cavaliers'] + ['Louisville Cardinals'] + ['Wake Forest Demon Deacons'] + 
                       ['North Carolina State Wolfpack'] + ['Syracuse Orange'] + ['Florida State Seminoles'] +
                       ['Boston College Eagles'] + ['Clemson Tigers'] + ['North Carolina Tar Heels'] + 
                      ['Miami (FL) Hurricanes'] + 
                       ['Duke Blue Devils'] + ['Virginia Tech Hokies'] + 
                       ['Pittsburgh Panthers'] + ['Georgia Tech Yellow Jackets'] + 
                       ['Virginia Cavaliers'] + ['Louisville Cardinals'] + ['Wake Forest Demon Deacons'] + 
                       ['North Carolina State Wolfpack'] + ['Syracuse Orange'] + ['Florida State Seminoles'] +
                       ['Boston College Eagles'] + ['Clemson Tigers'] + ['North Carolina Tar Heels'] + 
                      ['Miami (FL) Hurricanes'] + 
                       ['Duke Blue Devils'] + ['Notre Dame Fighting Irish'] + 
                       ['Virginia Tech Hokies'] + ['Georgia Tech Yellow Jackets'] + 
                       ['Virginia Cavaliers'] + ['Louisville Cardinals'] + ['Wake Forest Demon Deacons'] + 
                       ['North Carolina State Wolfpack'] + ['Syracuse Orange'] + ['Florida State Seminoles'] +
                       ['Boston College Eagles'] + ['Clemson Tigers'] + ['North Carolina Tar Heels'] + 
                      ['Miami (FL) Hurricanes'] + 
                       ['Duke Blue Devils'] + ['Notre Dame Fighting Irish'] + 
                       ['Pittsburgh Panthers'] + ['Virginia Tech Hokies'] + 
                       ['Virginia Cavaliers'] + ['Louisville Cardinals'] + ['Wake Forest Demon Deacons'] + 
                       ['North Carolina State Wolfpack'] + ['Syracuse Orange'] + ['Florida State Seminoles'] +
                       ['Boston College Eagles'] + ['Clemson Tigers'] + ['North Carolina Tar Heels'] + 
                      ['Miami (FL) Hurricanes'] + 
                       ['Duke Blue Devils'] + ['Notre Dame Fighting Irish'] + 
                       ['Pittsburgh Panthers'] + ['Georgia Tech Yellow Jackets'] + 
                       ['Virginia Tech Hokies'] + ['Louisville Cardinals'] + ['Wake Forest Demon Deacons'] + 
                       ['North Carolina State Wolfpack'] + ['Syracuse Orange'] + ['Florida State Seminoles'] +
                       ['Boston College Eagles'] + ['Clemson Tigers'] + ['North Carolina Tar Heels'] + 
                      ['Miami (FL) Hurricanes'] + 
                       ['Duke Blue Devils'] + ['Notre Dame Fighting Irish'] + 
                       ['Pittsburgh Panthers'] + ['Georgia Tech Yellow Jackets'] + 
                       ['Virginia Cavaliers'] + ['Virginia Tech Hokies'] + ['Wake Forest Demon Deacons'] + 
                       ['North Carolina State Wolfpack'] + ['Syracuse Orange'] + ['Florida State Seminoles'] +
                       ['Boston College Eagles'] + ['Clemson Tigers'] + ['North Carolina Tar Heels'] + 
                      ['Miami (FL) Hurricanes'] + 
                       ['Duke Blue Devils'] + ['Notre Dame Fighting Irish'] + 
                       ['Pittsburgh Panthers'] + ['Georgia Tech Yellow Jackets'] + 
                       ['Virginia Cavaliers'] + ['Louisville Cardinals'] + ['Virginia Tech Hokies'] + 
                       ['North Carolina State Wolfpack'] + ['Syracuse Orange'] + ['Florida State Seminoles'] +
                       ['Boston College Eagles'] + ['Clemson Tigers'] + ['North Carolina Tar Heels'] + 
                      ['Miami (FL) Hurricanes'] + 
                       ['Duke Blue Devils'] + ['Notre Dame Fighting Irish'] + 
                       ['Pittsburgh Panthers'] + ['Georgia Tech Yellow Jackets'] + 
                       ['Virginia Cavaliers'] + ['Louisville Cardinals'] + ['Wake Forest Demon Deacons'] + 
                       ['Virginia Tech Hokies'] + ['Syracuse Orange'] + ['Florida State Seminoles'] +
                       ['Boston College Eagles'] + ['Clemson Tigers'] + ['North Carolina Tar Heels'] + 
                      ['Miami (FL) Hurricanes'] + 
                       ['Duke Blue Devils'] + ['Notre Dame Fighting Irish'] + 
                       ['Pittsburgh Panthers'] + ['Georgia Tech Yellow Jackets'] + 
                       ['Virginia Cavaliers'] + ['Louisville Cardinals'] + ['Wake Forest Demon Deacons'] + 
                       ['North Carolina State Wolfpack'] + ['Virginia Tech Hokies'] + ['Florida State Seminoles'] +
                       ['Boston College Eagles'] + ['Clemson Tigers'] + ['North Carolina Tar Heels'] + 
                      ['Miami (FL) Hurricanes'] + 
                       ['Duke Blue Devils'] + ['Notre Dame Fighting Irish'] + 
                       ['Pittsburgh Panthers'] + ['Georgia Tech Yellow Jackets'] + 
                       ['Virginia Cavaliers'] + ['Louisville Cardinals'] + ['Wake Forest Demon Deacons'] + 
                       ['North Carolina State Wolfpack'] + ['Syracuse Orange'] + ['Virginia Tech Hokies'] +
                       ['Boston College Eagles'] + ['Clemson Tigers'] + ['North Carolina Tar Heels'] + 
                      ['Miami (FL) Hurricanes'] + 
                       ['Duke Blue Devils'] + ['Notre Dame Fighting Irish'] + 
                       ['Pittsburgh Panthers'] + ['Georgia Tech Yellow Jackets'] + 
                       ['Virginia Cavaliers'] + ['Louisville Cardinals'] + ['Wake Forest Demon Deacons'] + 
                       ['North Carolina State Wolfpack'] + ['Syracuse Orange'] + ['Florida State Seminoles'] +
                       ['Virginia Tech Hokies'] + ['Clemson Tigers'] + ['North Carolina Tar Heels'] + 
                      ['Miami (FL) Hurricanes'] + 
                       ['Duke Blue Devils'] + ['Notre Dame Fighting Irish'] + 
                       ['Pittsburgh Panthers'] + ['Georgia Tech Yellow Jackets'] + 
                       ['Virginia Cavaliers'] + ['Louisville Cardinals'] + ['Wake Forest Demon Deacons'] + 
                       ['North Carolina State Wolfpack'] + ['Syracuse Orange'] + ['Florida State Seminoles'] +
                       ['Boston College Eagles'] + ['Virginia Tech Hokies'] + ['North Carolina Tar Heels'] + 
                      ['Miami (FL) Hurricanes'] + 
                       ['Duke Blue Devils'] + ['Notre Dame Fighting Irish'] + 
                       ['Pittsburgh Panthers'] + ['Georgia Tech Yellow Jackets'] + 
                       ['Virginia Cavaliers'] + ['Louisville Cardinals'] + ['Wake Forest Demon Deacons'] + 
                       ['North Carolina State Wolfpack'] + ['Syracuse Orange'] + ['Florida State Seminoles'] +
                       ['Boston College Eagles'] + ['Clemson Tigers'] + ['Virginia Tech Hokies']})
testdf.head()

,Team,Team2
0,Virginia Tech Hokies,Miami (FL) Hurricanes
1,Virginia Tech Hokies,Duke Blue Devils
2,Virginia Tech Hokies,Notre Dame Fighting Irish
3,Virginia Tech Hokies,Pittsburgh Panthers
4,Virginia Tech Hokies,Georgia Tech Yellow Jackets


In [68]:
#Take the aggregated normalized values for just the independent variables
teamagg_norm2 = teamagg_norm[['DRB_Margin', 'FGP_Margin', '3FGP_Margin', 'AST_Margin', 'Fouls_Margin']]
teamagg_norm2.head()

,DRB_Margin,FGP_Margin,3FGP_Margin,AST_Margin,Fouls_Margin
Team,,,,,
Boston College Eagles,-0.808245,-0.622449,-0.146260,-0.327301,0.316352
Clemson Tigers,0.699262,0.771263,0.090829,0.692709,1.065326
Duke Blue Devils,0.981824,0.978718,-0.184413,0.851672,-0.972698
Florida State Seminoles,0.764175,0.482614,0.232979,0.851672,0.963425
Georgia Tech Yellow Jackets,-0.311088,0.415066,-0.407947,-0.790942,0.209356


In [69]:
#Merging the normalized values for Team1 in the new df
testdf = pd.merge(teamagg_norm2, testdf, on="Team", how = "left")
testdf.head()

,Team,DRB_Margin,FGP_Margin,3FGP_Margin,AST_Margin,Fouls_Margin,Team2
0,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Miami (FL) Hurricanes
1,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Duke Blue Devils
2,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Notre Dame Fighting Irish
3,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Pittsburgh Panthers
4,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Georgia Tech Yellow Jackets


In [70]:
#Updating the names for the teams
testdf = testdf.rename(columns = {'Team': 'Team1', 'Team2': 'Team'}, inplace = False)
testdf.head()

,Team1,DRB_Margin,FGP_Margin,3FGP_Margin,AST_Margin,Fouls_Margin,Team
0,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Miami (FL) Hurricanes
1,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Duke Blue Devils
2,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Notre Dame Fighting Irish
3,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Pittsburgh Panthers
4,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Georgia Tech Yellow Jackets


In [71]:
#Merging the normalized values for Team2 in the new df
testdf_comb = pd.merge(teamagg_norm2, testdf, on="Team", how='left')
testdf_comb.head()

,Team,DRB_Margin_x,FGP_Margin_x,3FGP_Margin_x,AST_Margin_x,Fouls_Margin_x,Team1,DRB_Margin_y,FGP_Margin_y,3FGP_Margin_y,AST_Margin_y,Fouls_Margin_y
0,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Clemson Tigers,0.699262,0.771263,0.090829,0.692709,1.065326
1,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Duke Blue Devils,0.981824,0.978718,-0.184413,0.851672,-0.972698
2,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Florida State Seminoles,0.764175,0.482614,0.232979,0.851672,0.963425
3,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Georgia Tech Yellow Jackets,-0.311088,0.415066,-0.407947,-0.790942,0.209356
4,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Louisville Cardinals,0.969491,0.519248,0.825572,0.805529,0.198656


In [72]:
#Renaming the columns
testdf_comb = testdf_comb.rename(columns = {'DRB_Margin_x': 'DRB_Margin_Team1', 'FGP_Margin_x': 'FGP_Margin_Team1', 
                                            '3FGP_Margin_x': '3FGP_Margin_Team1',
                                     'AST_Margin_x': 'AST_Margin_Team1', 'Fouls_Margin_x': 'Fouls_Margin_Team1', 
                                    'Team1': 'Team2', 'DRB_Margin_y': 'DRB_Margin_Team2',
                                     'FGP_Margin_y': 'FGP_Margin_Team2', '3FGP_Margin_y': '3FGP_Margin_Team2', 
                                    'AST_Margin_y': 'AST_Margin_Team2', 
                                    'Fouls_Margin_y': 'Fouls_Margin_Team2'}, inplace=False)
testdf_comb.head()

,Team,DRB_Margin_Team1,FGP_Margin_Team1,3FGP_Margin_Team1,AST_Margin_Team1,Fouls_Margin_Team1,Team2,DRB_Margin_Team2,FGP_Margin_Team2,3FGP_Margin_Team2,AST_Margin_Team2,Fouls_Margin_Team2
0,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Clemson Tigers,0.699262,0.771263,0.090829,0.692709,1.065326
1,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Duke Blue Devils,0.981824,0.978718,-0.184413,0.851672,-0.972698
2,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Florida State Seminoles,0.764175,0.482614,0.232979,0.851672,0.963425
3,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Georgia Tech Yellow Jackets,-0.311088,0.415066,-0.407947,-0.790942,0.209356
4,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Louisville Cardinals,0.969491,0.519248,0.825572,0.805529,0.198656


In [73]:
#Had to rename the team separate than the cell above
testdf_comb = testdf_comb.rename(columns = {'Team': 'Team1'})
testdf_comb.head()

,Team1,DRB_Margin_Team1,FGP_Margin_Team1,3FGP_Margin_Team1,AST_Margin_Team1,Fouls_Margin_Team1,Team2,DRB_Margin_Team2,FGP_Margin_Team2,3FGP_Margin_Team2,AST_Margin_Team2,Fouls_Margin_Team2
0,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Clemson Tigers,0.699262,0.771263,0.090829,0.692709,1.065326
1,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Duke Blue Devils,0.981824,0.978718,-0.184413,0.851672,-0.972698
2,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Florida State Seminoles,0.764175,0.482614,0.232979,0.851672,0.963425
3,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Georgia Tech Yellow Jackets,-0.311088,0.415066,-0.407947,-0.790942,0.209356
4,Boston College Eagles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,Louisville Cardinals,0.969491,0.519248,0.825572,0.805529,0.198656


In [74]:
#Move Team2 next to Team 1
t2 = testdf_comb.pop('Team2')
testdf_comb.insert(1, 'Team2', t2)
testdf_comb.head()

,Team1,Team2,DRB_Margin_Team1,FGP_Margin_Team1,3FGP_Margin_Team1,AST_Margin_Team1,Fouls_Margin_Team1,DRB_Margin_Team2,FGP_Margin_Team2,3FGP_Margin_Team2,AST_Margin_Team2,Fouls_Margin_Team2
0,Boston College Eagles,Clemson Tigers,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,0.699262,0.771263,0.090829,0.692709,1.065326
1,Boston College Eagles,Duke Blue Devils,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,0.981824,0.978718,-0.184413,0.851672,-0.972698
2,Boston College Eagles,Florida State Seminoles,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,0.764175,0.482614,0.232979,0.851672,0.963425
3,Boston College Eagles,Georgia Tech Yellow Jackets,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,-0.311088,0.415066,-0.407947,-0.790942,0.209356
4,Boston College Eagles,Louisville Cardinals,-0.808245,-0.622449,-0.14626,-0.327301,0.316352,0.969491,0.519248,0.825572,0.805529,0.198656


In [75]:
#Predict the Win Probabilities for each ACC matchup
testdf_comb['WinProb_Team1'] = lr.predict_proba(testdf_comb[ind_var_selected])[:,1]
testdf_comb.to_csv('PredictedWinProbs.csv')

In [76]:
#Aggregate the teams and average the predicted win probabilities as a rating
final = testdf_comb[['Team1', 'WinProb_Team1']].copy()
final = final.rename(columns = {'Team1': 'Team', 'WinProb_Team1': 'Rating'})
final = final.groupby(['Team'], as_index=False).agg('mean')
final = final.sort_values(by=['Rating'], ascending=False)
final.head()

,Team,Rating
12,Virginia Cavaliers,0.839542
8,North Carolina Tar Heels,0.806773
2,Duke Blue Devils,0.796457
1,Clemson Tigers,0.761242
3,Florida State Seminoles,0.750158


In [77]:
#Create a ranking column and set it as the index
ranking = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
final['Ranking'] = ranking
final.set_index('Ranking', inplace = True)
final.head(15)

,Team,Rating
Ranking,,
1,Virginia Cavaliers,0.839542
2,North Carolina Tar Heels,0.806773
3,Duke Blue Devils,0.796457
4,Clemson Tigers,0.761242
5,Florida State Seminoles,0.750158
6,Louisville Cardinals,0.738893
7,Virginia Tech Hokies,0.686918
8,Georgia Tech Yellow Jackets,0.567702
9,North Carolina State Wolfpack,0.488422


In [78]:
#Take the team rankings to csv
final=final.round(3)
final.to_csv('ACCRankings1819.csv')